In [1]:
%load_ext autoreload
%autoreload 2
    
from pathlib import Path
import metapack as mp
import geopandas as gpd
import pandas as pd
from auto_tqdm import tqdm 

doc = mp.jupyter.open_source_package()
doc.set_sys_path()
from  pylib import * 

ea_epsg = 2163 #US Equal Area projection

import logging
logging.basicConfig()

from pylib import lines_logger, points_logger
lines_logger.setLevel(logging.DEBUG)
points_logger.setLevel(logging.DEBUG)

pkg_root = Path(doc.path).parent
pkg = mp.open_package(pkg_root)
cache = open_cache(pkg)
pkg

import appnope



In [8]:
with appnope.nope_scope():
    build_lines(pkg)

INFO:pylib.lines:Split the input file
INFO:pylib.lines:   5328 splits keys
INFO:pylib.lines:Run the overlay process


Overlay Geohash:   0%|          | 0/5328 [00:00<?, ?it/s]

INFO:pylib.lines:Write the roads files


   5300 recombine keys


In [7]:
cache.get_df('recombine/603')

,osm_id,name,highway,waterway,aerialway,barrier,man_made,z_order,other_tags,band,row,west_value,cm_value,east_value,zone,epsg,us_state,cus_state,geometry
0,15228116,NaN,r,NaN,NaN,NaN,NaN,3,"""tiger:cfcc""=>""A41"",""tiger:county""=>""Fayette, ...",14,R,102W,99W,96W,14R,32614,1,1,"LINESTRING (-96.65926 29.97731, -96.65968 29.9..."
1,15228118,NaN,r,NaN,NaN,NaN,NaN,3,"""tiger:cfcc""=>""A41"",""tiger:tlid""=>""41998742"",""...",14,R,102W,99W,96W,14R,32614,1,1,"LINESTRING (-96.69559 29.98867, -96.69575 29.9..."
2,15228121,NaN,r,NaN,NaN,NaN,NaN,3,"""tiger:cfcc""=>""A41"",""tiger:tlid""=>""42000346"",""...",14,R,102W,99W,96W,14R,32614,1,1,"LINESTRING (-96.94546 29.76001, -96.94575 29.7..."
3,15228123,NaN,r,NaN,NaN,NaN,NaN,3,"""tiger:cfcc""=>""A41"",""tiger:tlid""=>""42028500:42...",14,R,102W,99W,96W,14R,32614,1,1,"LINESTRING (-96.85265 30.01835, -96.85524 30.0..."
4,15228124,NaN,r,NaN,NaN,NaN,NaN,3,"""tiger:cfcc""=>""A41"",""tiger:tlid""=>""42027285:42...",14,R,102W,99W,96W,14R,32614,1,1,"LINESTRING (-96.72261 30.09163, -96.72258 30.0..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7557,15243245,NaN,m,NaN,NaN,NaN,NaN,9,"""NHS""=>""Interstate"",""hgv""=>""designated"",""ref""=...",17,R,84W,81W,78W,17R,32617,1,1,"LINESTRING (-82.22383 28.57303, -82.22329 28.5..."
7558,15243362,NaN,m,NaN,NaN,NaN,NaN,29,"""NHS""=>""Interstate"",""hgv""=>""designated"",""ref""=...",17,R,84W,81W,78W,17R,32617,1,1,"LINESTRING (-82.21712 28.58045, -82.21781 28.5..."
7559,15243363,NaN,m,NaN,NaN,NaN,NaN,29,"""NHS""=>""Interstate"",""hgv""=>""designated"",""lit""=...",17,R,84W,81W,78W,17R,32617,1,1,"LINESTRING (-82.23722 28.52339, -82.23722 28.5..."
7560,15243364,NaN,m,NaN,NaN,NaN,NaN,29,"""NHS""=>""Interstate"",""hgv""=>""designated"",""lit""=...",17,R,84W,81W,78W,17R,32617,1,1,"LINESTRING (-82.23691 28.52290, -82.23691 28.5..."
